In [51]:
import os

os.getcwd()
os.chdir('/home/jecosine/Courses/Courses/COMP8800/Projects/CatBoostPlayground')
from allib.metrics import get_metrics
from allib.metrics.distance import AVAIL_DIST_METRICS
from allib.datasets import load_uci, AVAIL_DATASETS
import numpy as np
import pandas as pd

In [52]:
def load_distance(dsn: str, mn: str, suffix = "catonly"):
    fn = f"examples/dist_cache_1000/{dsn}/{mn}_{suffix}.npy"
    return np.load(fn)

def load_dataset(dsn: str):
    ds = load_uci(dsn)
    ds.with_preprocess(
        steps=[
            "sample_n", 
            "categorical_only",
            # "continuous_to_categorical",
            "remove_constant_columns", 
            "drop_duplicate_rows"
        ],  
        params_list=[
            {"n": 1000, "random_state": 0},
            # {"encode": "ordinal"},
            {},
            {}, 
            {}
        ], 
        in_place=True
    )
    for col in ds._data.columns:
        if ds._data[col].dtype == "category":
            ds._data[col] = ds._data[col].astype("category").cat.codes.astype("int")
    ds._data.reset_index(drop=True, inplace=True)
    
    return ds

DBN = "adult"
DM = "euclidean"

d = load_distance(DBN, DM, "catonly_wo_n")
ds = load_dataset(DBN)


In [5]:
from sklearn.metrics import pairwise_distances, euclidean_distances

In [53]:
data, label = ds._data, ds._label
batch = data.sample(10, random_state=0)
batch

,workclass,education,marital_status,occupation,relationship,race,sex,native_country
535,0,1,5,0,1,2,1,39
492,5,9,2,1,0,4,1,39
14,4,9,0,8,4,4,0,39
247,6,7,2,1,5,4,0,39
85,4,11,4,1,3,4,0,39
127,2,12,4,10,1,4,0,39
586,4,10,2,10,0,4,1,0
529,4,9,2,3,0,1,1,30
330,4,7,2,3,0,4,1,39
483,7,15,4,1,3,4,0,39


In [54]:
pairwise_distances(batch, metric="euclidean")

array([[ 0.        , 10.19803903, 13.52774926, 10.14889157, 11.26942767,
        15.19868415, 41.61730409, 13.45362405,  8.66025404, 16.        ],
       [10.19803903,  0.        ,  8.42614977,  5.56776436,  4.35889894,
        10.24695077, 40.04996879,  9.74679434,  3.        ,  7.34846923],
       [13.52774926,  8.42614977,  0.        ,  7.87400787,  8.36660027,
         6.4807407 , 39.33192088, 11.66190379,  7.07106781, 10.53565375],
       [10.14889157,  5.56776436,  7.87400787,  0.        ,  5.29150262,
        11.91637529, 40.5092582 , 11.3137085 ,  5.83095189,  8.54400375],
       [11.26942767,  4.35889894,  8.36660027,  5.29150262,  0.        ,
         9.48683298, 40.21193853, 10.58300524,  5.83095189,  5.        ],
       [15.19868415, 10.24695077,  6.4807407 , 11.91637529,  9.48683298,
         0.        , 39.17907605, 12.56980509,  9.16515139, 10.90871211],
       [41.61730409, 40.04996879, 39.33192088, 40.5092582 , 40.21193853,
        39.17907605,  0.        , 30.96772513

In [55]:
d[np.array(batch.index)[:, np.newaxis], np.array(batch.index)[np.newaxis, :]]

array([[ 0.        , 10.19803903, 13.52774926, 10.14889157, 11.26942767,
        15.19868415, 41.61730409, 13.45362405,  8.66025404, 16.        ],
       [10.19803903,  0.        ,  8.42614977,  5.56776436,  4.35889894,
        10.24695077, 40.04996879,  9.74679434,  3.        ,  7.34846923],
       [13.52774926,  8.42614977,  0.        ,  7.87400787,  8.36660027,
         6.4807407 , 39.33192088, 11.66190379,  7.07106781, 10.53565375],
       [10.14889157,  5.56776436,  7.87400787,  0.        ,  5.29150262,
        11.91637529, 40.5092582 , 11.3137085 ,  5.83095189,  8.54400375],
       [11.26942767,  4.35889894,  8.36660027,  5.29150262,  0.        ,
         9.48683298, 40.21193853, 10.58300524,  5.83095189,  5.        ],
       [15.19868415, 10.24695077,  6.4807407 , 11.91637529,  9.48683298,
         0.        , 39.17907605, 12.56980509,  9.16515139, 10.90871211],
       [41.61730409, 40.04996879, 39.33192088, 40.5092582 , 40.21193853,
        39.17907605,  0.        , 30.96772513

In [50]:
for dm in AVAIL_DIST_METRICS:
    d = load_distance(DBN, dm, "catonly_wo_n")
    if d.min() < 0:
        print(dm)

In [45]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

In [46]:
cosine_similarity([[1,1,1], [1,1,1], [0,0,0], [5,5,6]])

array([[1.        , 1.        , 0.        , 0.99611649],
       [1.        , 1.        , 0.        , 0.99611649],
       [0.        , 0.        , 0.        , 0.        ],
       [0.99611649, 0.99611649, 0.        , 1.        ]])

{'cityblock': <function sklearn.metrics.pairwise.manhattan_distances(X, Y=None, *, sum_over_features='deprecated')>,
 'cosine': <function sklearn.metrics.pairwise.cosine_distances(X, Y=None)>,
 'euclidean': <function sklearn.metrics.pairwise.euclidean_distances(X, Y=None, *, Y_norm_squared=None, squared=False, X_norm_squared=None)>,
 'haversine': <function sklearn.metrics.pairwise.haversine_distances(X, Y=None)>,
 'l2': <function sklearn.metrics.pairwise.euclidean_distances(X, Y=None, *, Y_norm_squared=None, squared=False, X_norm_squared=None)>,
 'l1': <function sklearn.metrics.pairwise.manhattan_distances(X, Y=None, *, sum_over_features='deprecated')>,
 'manhattan': <function sklearn.metrics.pairwise.manhattan_distances(X, Y=None, *, sum_over_features='deprecated')>,
 'precomputed': None,
 'nan_euclidean': <function sklearn.metrics.pairwise.nan_euclidean_distances(X, Y=None, *, squared=False, missing_values=nan, copy=True)>}